In [1]:

import os
import pandas as pd
import numpy as np

# ---------------------------------------------------------------------
# Question #1
# ---------------------------------------------------------------------


def get_assignment_names(grades):
    '''
    get_assignment_names takes in a dataframe like grades and returns
    a dictionary with the following structure:

    The keys are the general areas of the syllabus: lab, project,
    midterm, final, disc, checkpoint

    The values are lists that contain the assignment names of that type.
    For example the lab assignments all have names of the form labXX where XX
    is a zero-padded two digit number. See the doctests for more details.

    :Example:
    >>> grades_fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(grades_fp)
    >>> names = get_assignment_names(grades)
    >>> set(names.keys()) == {'lab', 'project', 'midterm', 'final', 'disc', 'checkpoint'}
    True
    >>> names['final'] == ['Final']
    True
    >>> 'project02' in names['project']
    True
    '''

    names = {'lab':[], 'project': [], 'midterm':['Midterm'], 'final':['Final'], 'disc':[], 'checkpoint':[]}

    for col in grades.columns:
        if ('lab' in col) and (len(col) == 5):
            names['lab'].append(col)
        elif ('project' in col) and (len(col) == 9):
            names['project'].append(col)
        elif ('discussion' in col) and (len(col) == 12):
            names['disc'].append(col)
        elif ('checkpoint' in col) and (len(col) == 22):
            names['checkpoint'].append(col)

    return names


# ---------------------------------------------------------------------
# Question #2
# ---------------------------------------------------------------------


def projects_total(grades):
    '''
    projects_total that takes in grades and computes the total project grade
    for the quarter according to the syllabus.
    The output Series should contain values between 0 and 1.

    :Example:
    >>> grades_fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(grades_fp)
    >>> out = projects_total(grades)
    >>> np.all((0 <= out) & (out <= 1))
    True
    >>> 0.7 < out.mean() < 0.9
    True
    '''
    names = get_assignment_names(grades)

    copy = grades.copy()
    project_scores = []

    #loop thru projects
    for project_name in names['project']:
        #check if theres a free response
        if project_name + '_free_response' in copy.columns:
            project_fr_name = project_name + '_free_response'
        else:
            project_fr_name = None
        #handle NaN values to get 0 score
        copy[project_name] = copy[project_name].fillna(0)
        if project_fr_name != None: #free response exists
            copy[project_fr_name] = copy[project_fr_name].fillna(0)

        #numerator
        if project_fr_name != None: #free response exists
            points = copy[project_name] + copy[project_fr_name]
        else:
            points = copy[project_name]
        #denominator
        if project_fr_name != None: #free response exists
            total_points = copy[project_name + ' - Max Points'] + copy[project_fr_name + ' - Max Points']
        else:
            total_points = copy[project_name + ' - Max Points']

        #compute scores for this project
        score = points/total_points
        project_scores.append(score)

    total_scores = sum(project_scores) / len(names['project']) #divided by number of projects
    return total_scores

# ---------------------------------------------------------------------
# Question # 3
# ---------------------------------------------------------------------


def last_minute_submissions(grades):
    """
    last_minute_submissions takes in the dataframe
    grades and a Series indexed by lab assignment that
    contains the number of submissions that were turned
    in on time by the student, yet marked 'late' by Gradescope.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = last_minute_submissions(grades)
    >>> isinstance(out, pd.Series)
    True
    >>> np.all(out.index == ['lab0%d' % d for d in range(1,10)])
    True
    >>> (out > 0).sum()
    8
    """

    late_tag = ' - Lateness (H:M:S)'

    names = get_assignment_names(grades)

    late_submissions = {}

    for lab in names['lab']:

        gradescope_late = grades.copy()

        gradescope_late = gradescope_late[gradescope_late[lab + late_tag] != '00:00:00']

        gradescope_late[lab + late_tag] = gradescope_late[lab + late_tag].str.slice(stop = 2)

        gradescope_late[lab + late_tag] = gradescope_late[lab + late_tag].astype(int)

        error_late = gradescope_late.loc[gradescope_late[lab + late_tag] < 9]

        late_submissions[lab] = len(error_late)

    series = pd.Series(late_submissions)
    return series


# ---------------------------------------------------------------------
# Question #4
# ---------------------------------------------------------------------

def lateness_penalty(col):
    """
    lateness_penalty takes in a 'lateness' column and returns
    a column of penalties according to the syllabus.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> col = pd.read_csv(fp)['lab01 - Lateness (H:M:S)']
    >>> out = lateness_penalty(col)
    >>> isinstance(out, pd.Series)
    True
    >>> set(out.unique()) <= {1.0, 0.9, 0.8, 0.5}
    True
    """

    #make series, starting with all 1.0 penalties
    penalties = pd.Series(1.0, index = col.index)

    #convert times to numbers
    col1 = col.str.contains('00:00:00')
    ontime_gc = col1[col1].index    
    col = col.drop(index = ontime_gc)
    
    if (col.empty == False): # there are lates to account for
        hours = col.str.split(':', 1, True)
        hours[0] = hours[0].astype(int)
        hours[1] = hours[1].str.contains('00:00')
        hours[0] = hours[0] + ~hours[1]
        hours = hours[0]

        #late 1 week (before 9 AM doesn't count)
        week_late = hours[hours.between(9, 168)]
        week_late_indices = week_late.index.values
        penalties[week_late_indices] = 0.9

        #late 2 weeks
        two_weeks = hours[hours.between(169, 336)]
        two_weeks_indices = two_weeks.index.values
        penalties[two_weeks_indices] = 0.8

        #beyond
        beyond = hours[hours >= 337]
        beyond_indices = beyond.index.values
        penalties[beyond_indices] = 0.5
    
    return penalties


# ---------------------------------------------------------------------
# Question #5
# ---------------------------------------------------------------------

def process_labs(grades):
    """
    process_labs that takes in a dataframe like grades and returns
    a dataframe of processed lab scores. The output should:
      * share the same index as grades,
      * have columns given by the lab assignment names (e.g. lab01,...lab10)
      * have values representing the lab grades for each assignment,
        adjusted for Lateness and scaled to a score between 0 and 1.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = process_labs(grades)
    >>> out.columns.tolist() == ['lab%02d' % x for x in range(1,10)]
    True
    >>> np.all((0.65 <= out.mean()) & (out.mean() <= 0.90))
    True
    """

    #do we need to handle NaNs?
    grades = grades.replace({np.nan: 0})
    
    new_df = pd.DataFrame()
    lab_titles = get_assignment_names(grades)['lab']
    lateness =  ' - Lateness (H:M:S)'
    max_p = ' - Max Points'
    
    for col in lab_titles:
        new_df[col] = (grades[col] / grades[col + max_p]) * lateness_penalty(grades[col + lateness])
        
    return new_df


# ---------------------------------------------------------------------
# Question #6
# ---------------------------------------------------------------------

def lab_total(processed):
    """
    lab_total takes in dataframe of processed assignments (like the output of
    Question 5) and computes the total lab grade for each student according to
    the syllabus (returning a Series).

    Your answers should be proportions between 0 and 1.

    :Example:
    >>> cols = 'lab01 lab02 lab03'.split()
    >>> processed = pd.DataFrame([[0.2, 0.90, 1.0]], index=[0], columns=cols)
    >>> np.isclose(lab_total(processed), 0.95).all()
    True
    """

    # clean out the nans (turn to 0)
    processed = processed.replace({np.nan: 0})
    n = len(processed.columns) - 1
    
    lowest = processed.min(axis = 1)
    summed = processed.sum(axis = 1)
    total = summed - lowest
    
    return total / n

# ---------------------------------------------------------------------
# Question # 7
# ---------------------------------------------------------------------

# checkpoints 
def checkpoint_total(grades):
    grades = grades.replace({np.nan: 0})
    columns = get_assignment_names(grades)
    col_name = ' - Max Points'
    checkpoint_scores = []
    
    for col in columns['checkpoint']:
        checkpoint_scores.append(grades[col] / grades[col + col_name])
        
    total_checkpoint = sum(checkpoint_scores) / len(columns['checkpoint'])
    
    return total_checkpoint


# discussion
def discussion_total(grades):
    grades = grades.replace({np.nan: 0})
    columns = get_assignment_names(grades)
    col_name = ' - Max Points'
    discussion_scores = []
    
    for col in columns['disc']:
        discussion_scores.append(grades[col] / grades[col + col_name])
        
    total_discussion = sum(discussion_scores) / len(columns['disc'])
    
    return total_discussion


def total_points(grades):
    """
    total_points takes in grades and returns the final
    course grades according to the syllabus. Course grades
    should be proportions between zero and one.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = total_points(grades)
    >>> np.all((0 <= out) & (out <= 1))
    True
    >>> 0.7 < out.mean() < 0.9
    True
    """

    grades = grades.replace({np.nan: 0})
    
    # lab total
    processed = process_labs(grades)
    lab_scores = lab_total(processed)
    
    # project total
    project_scores = projects_total(grades)
    
    # checkpoint total
    checkpoint_scores = checkpoint_total(grades)

    # discussion total
    discussion_scores = discussion_total(grades)

    # exam total
    col_name = ' - Max Points'

    midterm_scores = grades['Midterm'] / grades['Midterm' + col_name]
    final_scores = grades['Final'] / grades['Final' + col_name]
    
    return (lab_scores * 0.2) + (project_scores * 0.3) + (checkpoint_scores * 0.025) + (discussion_scores * 0.025) + (midterm_scores * 0.15) + (final_scores * 0.3)


def final_grades(total):
    """
    final_grades takes in the final course grades
    as above and returns a Series of letter grades
    given by the standard cutoffs.

    :Example:
    >>> out = final_grades(pd.Series([0.92, 0.81, 0.41]))
    >>> np.all(out == ['A', 'B', 'F'])
    True
    """

    letter = total.where(~total.between(0.9, 1), 'A')
    letter = letter.where(~total.between(0.8, 0.9), 'B')
    letter = letter.where(~total.between(0.7, 0.8), 'C')
    letter = letter.where(~total.between(0.6, 0.7), 'D')
    letter = letter.where(~total.between(0, 0.6), 'F')
    
    return letter


def letter_proportions(grades):
    """
    letter_proportions takes in the dataframe grades
    and outputs a Series that contains the proportion
    of the class that received each grade.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = letter_proportions(grades)
    >>> np.all(out.index == ['B', 'C', 'A', 'D', 'F'])
    True
    >>> out.sum() == 1.0
    True
    """

    total = total_points(grades)
    letter = final_grades(total)
    
    distribution = letter.value_counts()
    
    return distribution / grades.shape[0]

# ---------------------------------------------------------------------
# Question # 8
# ---------------------------------------------------------------------

def simulate_pval(grades, N):
    """
    simulate_pval takes in the number of
    simulations N and grades and returns
    the likelihood that the grade of sophomores
    was no better on average than the class
    as a whole (i.e. calculate the p-value).

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = simulate_pval(grades, 100)
    >>> 0 <= out <= 0.1
    True
    """
    
    sophomores = grades.loc[grades['Level'] == 'SO']
    total_s = total_points(sophomores)
    test_statistic = total_s.mean()
    
    n_means = []
    mean_grades = total_points(grades).mean()

    for i in range(N):
        sample = grades.sample(sophomores.shape[0], replace = False)
        sample_total = total_points(sample)
        new_avg = sample_total.mean()
        n_means.append(new_avg)

    return np.count_nonzero(np.array(n_means) >= test_statistic) / N


# ---------------------------------------------------------------------
# Question # 9
# ---------------------------------------------------------------------

def unwind(columns):
    columns.values()
    cols = []
    
    for i in columns.values():
        for j in i:
            cols.append(j)
            
    return cols

def total_points_with_noise(grades):
    """
    total_points_with_noise takes in a dataframe like grades,
    adds noise to the assignments as described in notebook, and returns
    the total scores of each student calculated with noisy grades.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = total_points_with_noise(grades)
    >>> np.all((0 <= out) & (out <= 1))
    True
    >>> 0.7 < out.mean() < 0.9
    True
    """

    num_rows = grades.shape[0]
    num_cols = grades.shape[1]
    random_amt = pd.DataFrame(np.random.normal(0, 0.02, size=(num_rows, num_cols)))
    
    noise = grades.copy()
    columns = get_assignment_names(grades)
    n = 0
    
    for col in unwind(columns):
        noise[col] = np.clip(noise[col] + random_amt[n], 0, 100)
        n += 1
    
    return total_points(noise)


# ---------------------------------------------------------------------
# Question #10
# ---------------------------------------------------------------------

def short_answer():
    """
    short_answer returns (hard-coded) answers to the
    questions listed in the notebook. The answers should be
    given in a list with the same order as questions.

    :Example:
    >>> out = short_answer()
    >>> len(out) == 5
    True
    >>> len(out[2]) == 2
    True
    >>> 50 < out[2][0] < 100
    True
    >>> 0 < out[3] < 1
    True
    >>> isinstance(out[4], bool)
    True
    """

    return [1.0174066278123561e-05, 100, [0, 100.0], 0, True]

# ---------------------------------------------------------------------
# DO NOT TOUCH BELOW THIS LINE
# IT'S FOR YOUR OWN BENEFIT!
# ---------------------------------------------------------------------


# Graded functions names! DO NOT CHANGE!
# This dictionary provides your doctests with
# a check that all of the questions being graded
# exist in your code!

GRADED_FUNCTIONS = {
    'q01': ['get_assignment_names'],
    'q02': ['projects_total'],
    'q03': ['last_minute_submissions'],
    'q04': ['lateness_penalty'],
    'q05': ['process_labs'],
    'q06': ['lab_total'],
    'q07': ['total_points', 'final_grades', 'letter_proportions'],
    'q08': ['simulate_pval'],
    'q09': ['total_points_with_noise'],
    'q10': ['short_answer']
}


def check_for_graded_elements():
    """
    >>> check_for_graded_elements()
    True
    """

    for q, elts in GRADED_FUNCTIONS.items():
        for elt in elts:
            if elt not in globals():
                stmt = "YOU CHANGED A QUESTION THAT SHOULDN'T CHANGE! \
                In %s, part %s is missing" %(q, elt)
                raise Exception(stmt)

    return True


In [1]:
# DSC 80: Project 01

### Checkpoint Due Date: Thursday April 9, 11:59:59 PM (Questions 1-4)
### Due Date: Thursday, April 16, 11:59:59 PM

In [102]:
grades = grades.replace({np.nan: 0})
grades.loc[23]['lab02']

0.0

---
# Instructions

This Jupyter Notebook contains the statements of the problems and provides code and markdown cells to display your answers to the problems.  
* Like the lab, your coding work will be developed in the accompanying `project01.py` file, that will be imported into the current notebook. This code will be autograded.
* **For the checkpoint, turn in questions 1-4**

**Do not change the function names in the `*.py` file**
- The functions in the `*.py` file are how your assignment is graded, and they are graded by their name. The dictionary at the end of the file (`GRADED FUNCTIONS`) contains the "grading list". The final function in the file allows your doctests to check that all the necessary functions exist.
- If you changed something you weren't supposed to, just use git to revert!

**Tips for developing in the .py file**:
- Do not change the function names in the starter code; grading is done using these function names.
- Do not change the docstrings in the functions. These are there to tell you if your work is on the right track!
- You are **encouraged to write your own additional functions** to solve the questions! 
    - Developing in python usually consists of larger files, with many short functions.
    - You may write your other functions in an additional `.py` file that you import in `project01.py` -- however, be sure to upload these to gradescope as well!
- Always document your code!

**Tips for testing the correctness of your answers!**
Once you have your work saved in the .py file, you should import the `project01` to test your function out in the notebook. In the notebook you should inspect/analyze the output to assess its correctness!
* Run your functions on the main dataset (`grades`) and ask yourself if the output *looks correct.*
* Run your functions on very small datasets (e.g. 1-5 row table), calculate the expected response by hand, and see if the function output matches (this *is* unit-testing your code with data).
* Run your functions on (large and small) samples of the dataset `grades` (with and without replacement). Does your code break? Or does it still run as expected.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import project01 as proj

In [4]:
%matplotlib inline
import pandas as pd
import numpy as np

import os

# The Other Side of Gradescope

The file contains the grade-book from a fictional data science course with 535 students. 

**Note: this dataset is synthetically generated; it does not contain real student grades.**

In this project, you will:
1. clean and process the data to compute total course grades according to a fictional syllabus (below),
2. qualitatively understand how students did in the course,
3. understand how student grades vary with small changes in performance on each assignment.

---

The course syllabus is as follows:

* Lab assignments 
    - Each are worth the same amount, regardless of each lab's raw point total.
    - The lowest lab is dropped.
    - Each lab may be revised for one week after submission for a 10% penalty, for two weeks after submission for a 20% penalty, and beyond that for a 50% penalty. Such revisions are reflected in the `Lateness` columns in the gradebook.
    - Labs are 20% of the total grade.
* Projects 
    - Each project consists of an autograded portion, and *possibly* a free response portion.
    - The total points for a single project consist of the sum of the raw score of the two portions.
    - Each are worth the same amount, regardless of each project's raw point total.
    - Projects are 30% of the total grade.
* Checkpoints
    - Project checkpoints are worth 2.5% of the total grade.
* Discussion
    - Discussion notebooks are worth 2.5% of the total grade.
* Exams
    - The midterm is worth 15% of the total grade.
    - The final is worth 30% of the total grade.


### A note on generalization

You may assume that your code will only need to work on a gradebook for a class with the syllabus given above. That is, you may assume that the dataframe `grades` looks like the given one in `data/grades.csv`.

However, such a class:
1. may have a different numbers of labs, projects, discussions, and project checkpoints.
2. may have a different number of students.

You may assume the course components and the naming conventions are as given in the data file.

The dataset was generated by Gradescope; you must attempt to reason about the data as given using what you know as a student who uses Gradescope.

### A note on 'putting everything together'

The goal of this project is to create and assess final grades for a fictional course; if anything, the process is broken down into functions for your convenience and guidance. Here are a few remarks and tips for approaching the projects:
1. If you are having trouble figuring out what a question is asking you to do, look at the big picture and try to understand what the current step is doing to contribute to this big picture. This may clarify what's being asked!
1. These questions intentionally build off of each other and the final result matters! In fact, you can 'get a question correct', but only receive partial credit on it because a previous answer was wrong.
    - Credit for a question will typically receive partial credit based on *how close* your answer is to correct (as well as some credit for a solution in the correct form). 
    - You should try to assess your answer to each question based on what you understand of the data. This might involve writing extensive code (that isn't turned in) just to check your work! Suggestions on checking your work are given in the assignment, but you should also think of your own ways of checking your work.
    - As you do this project, think about the data from the perspective of the student (which should be easy to do!)

In [5]:
grades_fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(grades_fp)

### Getting started: enumerating the assignments

First, you will list all the 'assignment names' and what part of the syllabus to which they belong.

**Question 1:**

Create a function `get_assignment_names` that takes in a dataframe like `grades` and returns a dictionary with the following structure:
- The keys are the general areas of the syllabus: `lab, project, midterm, final, disc, checkpoint`
- The values are lists that contain the assignment names of that type. For example the lab assignments all have names of the form `labXX` where `XX` is a zero-padded two digit number. See the doctests for more details.

In [6]:
def get_assignment_names(df):
    
    names = {'lab':[], 'project': [], 'midterm':['Midterm'], 'final':['Final'], 'disc':[], 'checkpoint':[]}
    
    
    #is it always up to 10?
    for col in df.columns:
        if ('lab' in col) and (len(col) == 5):
            names['lab'].append(col)
        elif ('project' in col) and (len(col) == 9):
            names['project'].append(col)
        elif ('discussion' in col) and (len(col) == 12):
            names['disc'].append(col)
        elif ('checkpoint' in col) and (len(col) == 22):
            names['checkpoint'].append(col)
            
    return names

In [24]:
grades.head()

,PID,College,Level,lab01,lab01 - Max Points,lab01 - Lateness (H:M:S),lab02,lab02 - Max Points,lab02 - Lateness (H:M:S),project01,...,discussion07 - Lateness (H:M:S),discussion08,discussion08 - Max Points,discussion08 - Lateness (H:M:S),discussion09,discussion09 - Max Points,discussion09 - Lateness (H:M:S),discussion10,discussion10 - Max Points,discussion10 - Lateness (H:M:S)
0,A14721419,SI,JR,99.0,100.0,00:00:00,86.0,100.0,00:00:00,75.0,...,00:00:00,10.0,10,00:00:00,10.0,10,780:01:28,10.0,10,00:00:00
1,A14883274,TH,JR,98.0,100.0,00:00:00,52.0,100.0,00:00:00,53.0,...,669:12:21,7.0,10,00:00:00,7.0,10,00:00:00,8.0,10,00:00:00
2,A14164800,SI,SR,86.0,100.0,00:00:00,45.0,100.0,00:00:00,44.0,...,00:00:00,6.0,10,00:04:51,6.0,10,00:00:00,7.0,10,00:00:00
3,A14847419,TH,JR,100.0,100.0,00:00:00,100.0,100.0,00:00:00,78.0,...,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00
4,A14162943,SI,JR,66.0,100.0,00:00:00,33.0,100.0,00:00:00,42.0,...,00:00:00,5.0,10,00:00:00,5.0,10,00:00:00,6.0,10,00:00:00


In [8]:
pd.isnull(grades.loc[64].project01)

False

In [9]:
grades.loc[62].project01

nan

### Computing project grades

**Question 2**

Compute the total score for the project portion of the course according to the syllabus. Create a function `projects_total` that takes in `grades` and computes the total project grade for the quarter according to the syllabus. The output Series should contain values between 0 and 1.

*Note*: Don't forget to properly handle students who didn't turn in assignments! (Use your experience and common sense).

*Note:* To check your work, try (1) calculating the score for a few types of students by hand, and (2) calculate the statistics for the class performance on each individual course project, making sure they look reasonable.

In [10]:
#need to handle students who don't turn it in
def projects_total(grades):
    
    names = get_assignment_names(grades)

    copy = grades.copy()
    project_scores = [] 
    
    #loop thru projects
    for project_name in names['project']:
        
        #check if theres a free response
        if project_name + '_free_response' in copy.columns:
            project_fr_name = project_name + '_free_response'
        else:
            project_fr_name = None
            
        #handle NaN values to get 0 score
        copy[project_name] = copy[project_name].fillna(0)
        if project_fr_name != None: #free response exists
            copy[project_fr_name] = copy[project_fr_name].fillna(0)
            
            
        #numerator
        if project_fr_name != None: #free response exists
            points = copy[project_name] + copy[project_fr_name]
        else:
            points = copy[project_name]
        
        #denominator
        if project_fr_name != None: #free response exists
            total_points = copy[project_name + ' - Max Points'] + copy[project_fr_name + ' - Max Points']
        else:
            total_points = copy[project_name + ' - Max Points']
            
        #compute scores for this project
        score = points/total_points
        project_scores.append(score)
        
    
    total_scores = sum(project_scores) / len(names['project']) #divided by number of projects
    return total_scores

In [11]:
projects_total(grades)

0      0.900000
1      0.759333
2      0.673333
3      0.952667
4      0.718667
         ...   
530    0.949333
531    0.846667
532    0.837333
533    0.797333
534    0.948000
Length: 535, dtype: float64

In [12]:
out = projects_total(grades)
#np.all((0 <= out) & (out <= 1))

0.7 < out.mean() < 0.9

True

### Computing lab grades

Now, you will clean and process the lab grades, which is a little more complicated. To do this, you will develop functions that:
- 'normalize' the grades, 
- adjust for late submissions, 
- drop the lowest lab grade, and 
- creates a total lab score for each student.

**Question 3**

Unfortunately, Gradescope sometimes experiences a delay in registering when an assignment is submitted during "periods of heavy usage" (i.e. near a submission deadline). You need to assess when a student's assignment was actually turned in on time, even if Gradescope did not process it in time. To do this, it is helpful to know:
* Every late submission has to be submitted by a TA (late submissions are turned off).
* TAs never submitted a late assignment "just after" the deadline. 
* The deadlines were at midnight and students had to come to staff hours to late-submit their assignment.

Create a function `last_minute_submissions` that takes in the dataframe `grades` and outputs the number of submissions on each assignment that were turned in on time by the student, yet marked 'late' by Gradescope. See the doctest for more details.

*Note:* You have to figure out what truly is a late submission by looking at the data and understanding the facts about the data generating process above. There is some ambiguity in finding which submissions are truly late; you will *make a best guess for a threshold* by looking at this dataset. This question is about 'cleaning' a messy 'data recording process'.

In [13]:
def last_minute_submissions(grades):
    
    late_tag = ' - Lateness (H:M:S)'
        
    names = get_assignment_names(grades)
    
    late_submissions = {}
    
    for lab in names['lab']:
        
        gradescope_late = grades.copy()
        
        gradescope_late = gradescope_late[gradescope_late[lab + late_tag] != '00:00:00']
    
        gradescope_late[lab + late_tag] = gradescope_late[lab + late_tag].str.slice(stop = 2)
                
        gradescope_late[lab + late_tag] = gradescope_late[lab + late_tag].astype(int)
        
        error_late = gradescope_late.loc[gradescope_late[lab + late_tag] < 9]
        
        late_submissions[lab] = len(error_late)
        
    series = pd.Series(late_submissions)  
    return series

In [14]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = last_minute_submissions(grades)
(out > 0).sum()


8

**Question 4**

Now you need to adjust the lab grades for late submissions -- however, you need to take into account your investigation in the previous question, since students shouldn't be penalized by a bug in Gradescope!

Create a function `lateness_penalty` that takes in a 'Lateness' column and returns a column of penalties (represented by the values `1.0,0.9,0.8,0.5` according to the syllabus). Only *truly* late submissions should be counted as late.

*Note*: For the purpose of this project, we will only be calculating lateness for labs. There is no penalty for lateness for projects, discussions, nor checkpoints.

In [44]:
def lateness_penalty(col):
    
    #make series, starting with all 1.0 penalties
    penalties = pd.Series(1.0, index = col.index)

    #convert times to numbers
    col1 = col.str.contains('00:00:00')
    ontime_gc = col1[col1].index    
    col = col.drop(index = ontime_gc)
    
    hours = col.str.split(':', 1, True)
    hours[0] = hours[0].astype(int)
    hours[1] = hours[1].str.contains('00:00')
    hours[0] = hours[0] + ~hours[1]
    hours = hours[0]

    #late 1 week (before 9 AM doesn't count)
    week_late = hours[hours.between(9, 168)]
    week_late_indices = week_late.index.values
    penalties[week_late_indices] = 0.9

    #late 2 weeks
    two_weeks = hours[hours.between(169, 336)]
    two_weeks_indices = two_weeks.index.values
    penalties[two_weeks_indices] = 0.8

    #beyond
    beyond = hours[hours >= 337]
    beyond_indices = beyond.index.values
    penalties[beyond_indices] = 0.5

    return penalties

In [16]:
fp = os.path.join('data', 'grades.csv')
col = pd.read_csv(fp)['lab01 - Lateness (H:M:S)']
out = lateness_penalty(col)
set(out.unique()) <= {1.0, 0.9, 0.8, 0.5}

NameError: name 'hour' is not defined

In [17]:
1 week (168 hrs) - 10%
2 weeks (169-336 hrs) - 20%
beyond (337+hrs) - 50%

SyntaxError: invalid syntax (<ipython-input-17-f5fd4790c37f>, line 1)

In [ ]:
col = grades['lab01 - Lateness (H:M:S)']

#make series, starting with all 1.0 penalties
penalties = pd.Series(1.0, index = col.index)

#convert times to numbers
hours = col.str.slice(stop = 2)
hours = hour.astype(int)

#late 1 week (before 9 hours doesnt count)
week_late = hours[hours.between(9, 168)]
week_late_indices = week_late.index.values
penalties[week_late_indices] = 0.9

#late 2 weeks
two_weeks = hours[hours.between(169, 336)]
two_weeks_indices = two_weeks.index.values
penalties[two_weeks_indices] = 0.8

#beyond
beyond = hours[hours >= 337]
beyond_indices = beyond.index.values
penalties[beyond_indices] = 0.5

penalties


In [18]:
test_col = pd.Series([10, 10, 10, 10, 10, 10], index = [0, 1, 2, 3, 4, 5])
i_to_replace = np.array([1, 2, 5])
replaced = 100
test_col[i_to_replace] = 100
test_col


0     10
1    100
2    100
3     10
4     10
5    100
dtype: int64

**Question 5**

Create a function `process_labs` that takes in a dataframe like `grades` and returns a dataframe of processed lab scores. The output should:
* share the same index as `grades`,
* have columns given by the lab assignment names (e.g. `lab01,...lab10`)
* have values representing the lab grades for each assignment, adjusted for Lateness and scaled to a score between 0 and 1.

In [86]:
series = grades['lab01']
index_to_replace = series[series > 90].index.values
#series.replace(index_to_replace, 'A')
#series[series > 90]
#series.loc[index_to_replace] = 'A'
index_to_replace



TypeError: '>' not supported between instances of 'str' and 'int'

In [108]:
def process_labs(grades):
    names = get_assignment_names(grades)
    processed_labs = pd.DataFrame()

    for lab in names['lab']:
        lab_grades = grades[lab] / grades[lab + ' - Max Points']
        lab_times = grades[lab + ' - Lateness (H:M:S)']
        lab_grades_adjusted = (lateness_penalty(lab_times) * lab_grades) 
        processed_labs[lab] = lab_grades_adjusted

    return processed_labs

In [56]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = process_labs(grades)
np.all((0.65 <= out.mean()) & (out.mean() <= 0.90))


True

In [48]:
grades['lab09'].mean()

42.39463601532567

**Question 6**

Create a function `lab_total` that takes in dataframe of processed assignments (like the output of Question 5) and computes the total lab grade for each student according to the syllabus (returning a Series). Your answers should be proportions between 0 and 1. For example, if there are only 3 labs, and a student received scores of {80%,90%,100%}, then the total score would be 0.95.

*Note*: Don't forget to properly handle students who didn't turn in assignments! (Use your experience and common sense).

In [187]:
grades.head()

,PID,College,Level,lab01,lab01 - Max Points,lab01 - Lateness (H:M:S),lab02,lab02 - Max Points,lab02 - Lateness (H:M:S),project01,...,discussion07 - Lateness (H:M:S),discussion08,discussion08 - Max Points,discussion08 - Lateness (H:M:S),discussion09,discussion09 - Max Points,discussion09 - Lateness (H:M:S),discussion10,discussion10 - Max Points,discussion10 - Lateness (H:M:S)
0,A14721419,SI,JR,99.0,100.0,00:00:00,86.0,100.0,00:00:00,75.0,...,00:00:00,10.0,10,00:00:00,10.0,10,780:01:28,10.0,10,00:00:00
1,A14883274,TH,JR,98.0,100.0,00:00:00,52.0,100.0,00:00:00,53.0,...,669:12:21,7.0,10,00:00:00,7.0,10,00:00:00,8.0,10,00:00:00
2,A14164800,SI,SR,86.0,100.0,00:00:00,45.0,100.0,00:00:00,44.0,...,00:00:00,6.0,10,00:04:51,6.0,10,00:00:00,7.0,10,00:00:00
3,A14847419,TH,JR,100.0,100.0,00:00:00,100.0,100.0,00:00:00,78.0,...,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00
4,A14162943,SI,JR,66.0,100.0,00:00:00,33.0,100.0,00:00:00,42.0,...,00:00:00,5.0,10,00:00:00,5.0,10,00:00:00,6.0,10,00:00:00


In [ ]:
#in sophs: labs 3, 5, 7, 8
#in grades.head(): labs 1, 2, 5, 8, 9

In [152]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp) 3, 5, 7, 8

In [193]:
sophs = grades[grades['Level'] == 'SO']
x = lateness_penalty(sophs['lab03 - Lateness (H:M:S)'])
x

KeyError: 0

In [192]:
'asdf' / 12.3

TypeError: unsupported operand type(s) for /: 'str' and 'float'

In [194]:
col = sophs['lab03 - Lateness (H:M:S)']
hours = col.str.split(':', 1, True)
hours[0] = hours[0].astype(int)
hours[0]

20     0
27     0
60     0
64     0
80     0
83     0
91     0
92     0
96     0
97     0
99     0
111    0
121    0
155    0
157    0
161    0
172    0
173    0
188    0
204    0
209    0
225    0
245    0
246    0
254    0
274    0
290    0
299    0
307    0
312    0
317    0
320    0
321    0
323    0
327    0
330    0
334    0
364    0
366    0
367    0
375    0
380    0
383    0
387    0
388    0
395    0
409    0
417    0
433    0
434    0
441    0
473    0
477    0
487    0
490    0
505    0
515    0
520    0
531    0
Name: 0, dtype: int32

### Putting it together

**Question 7**

Finally, you need to create the final course grades. To do this, you will add up the total of each course component according to the weights given in the syllabus. 

* Create a function `total_points` that takes in `grades` and returns the final course grades according to the syllabus. Course grades should be proportions between zero and one.
* Create a function `final_grades` that takes in the final course grades as above and returns a Series of letter grades given by the standard cutoffs (`A >= .90`, `.90 > B >= .80`, `.80 > C >= .70`, `.70 > D >= .60`, `.60 > F`). You should not use rounding to determining the letter grades.
* Create a function `letter_proportions` which takes in the dataframe `grades` and outputs a Series that contains the proportion of the class that received each grade. (This question requires you to put everything together).
* The indices should be ordered by the proportion of the class that receives that grade, from largest to smallest.

*Note 1*: Don't repeat yourself when computing the checkpoint and discussion portions of the course.

*Note 2*: Only the lab portion of the course accounts for late assignments; you may assume all assignments in other portions are turned in without penalty.

*Note 3*: These values should add up to exactly 1.0. If you are getting something close such as 0.99999, that means there is a slight issue with your code from above. 

To check your work, verify the course grade distribution and relevant statistics! Do the work by hand for a few students.

In [57]:
def checkpoint_total(grades):
    
    

,PID,College,Level,lab01,lab01 - Max Points,lab01 - Lateness (H:M:S),lab02,lab02 - Max Points,lab02 - Lateness (H:M:S),project01,...,discussion07 - Lateness (H:M:S),discussion08,discussion08 - Max Points,discussion08 - Lateness (H:M:S),discussion09,discussion09 - Max Points,discussion09 - Lateness (H:M:S),discussion10,discussion10 - Max Points,discussion10 - Lateness (H:M:S)
0,A14721419,SI,JR,99.0,100.0,00:00:00,86.0,100.0,00:00:00,75.0,...,00:00:00,10.0,10,00:00:00,10.0,10,780:01:28,10.0,10,00:00:00
1,A14883274,TH,JR,98.0,100.0,00:00:00,52.0,100.0,00:00:00,53.0,...,669:12:21,7.0,10,00:00:00,7.0,10,00:00:00,8.0,10,00:00:00
2,A14164800,SI,SR,86.0,100.0,00:00:00,45.0,100.0,00:00:00,44.0,...,00:00:00,6.0,10,00:04:51,6.0,10,00:00:00,7.0,10,00:00:00
3,A14847419,TH,JR,100.0,100.0,00:00:00,100.0,100.0,00:00:00,78.0,...,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00
4,A14162943,SI,JR,66.0,100.0,00:00:00,33.0,100.0,00:00:00,42.0,...,00:00:00,5.0,10,00:00:00,5.0,10,00:00:00,6.0,10,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,A14490387,SI,JR,100.0,100.0,47:26:10,82.0,100.0,00:00:00,78.0,...,00:00:00,10.0,10,12:08:58,10.0,10,00:00:00,10.0,10,00:00:00
531,A14088257,SI,SO,100.0,100.0,00:00:00,86.0,100.0,00:00:00,72.0,...,00:00:00,9.0,10,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00
532,A14847419,WA,JR,87.0,100.0,00:00:00,90.0,100.0,00:00:00,66.0,...,00:00:00,9.0,10,00:00:00,9.0,10,00:00:00,9.0,10,00:00:00
533,A14513929,TH,SR,84.0,100.0,00:00:00,83.0,100.0,00:00:00,62.0,...,00:00:00,9.0,10,00:00:00,9.0,10,419:06:41,8.0,10,00:00:00


### Do Sophomores get better grades?

**Question 8**

You notice that students who are sophomores on average did better in the class (if you can't verify this, you should go back and check your work!). Is this difference significant, or just due to noise?

Perform a hypothesis test, assessing likelihood of the null hypothesis: 
> "sophomores earn grades that are roughly equal on average to the rest of the class."


Create a function `simulate_pval` which takes in the number of simulations `N` and `grades` and returns the the likelihood that the grade of sophomores was no better on average than the class as a whole (i.e. calculate the p-value).

*Note:* To check your work, plot the sampling distribution and the observation. Do these values look reasonable?

In [89]:
sophs = grades[grades['Level'] == 'SO']
total_grades(sophs)

NameError: name 'total_grades' is not defined

In [33]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)

### What is the true distribution of grades?

The gradebook for this class only reflects one particular instance of each student's performance, subject to the effects of all the little events and hiccups that occurred throughout the quarter. Might you have done better on the midterm had your roommate kept you up all night with their coughing? Wasn't it lucky that the example you were studying just before the final happened to appear on the exam?

**Question 9**

This question will simulate these '(un)lucky, random events' by adding or subtracting random amounts to each assignment before calculating the final grades. These 'random amounts' will be drawn from a Gaussian distribution of mean 0 and a std deviation 0.02:
```
np.random.normal(0, 0.02, size=(num_rows, num_cols))
```
Intuitively, such a model says that random events may bump up or down a given grade (given as a proportion):
- which on average has no effect on the class as a whole (mean 0),
- which not uncommonly might perturb a grade by 2% (std dev 0.02).

Create a function `total_points_with_noise` that takes in a dataframe like `grades`, adds noise to the assignments as described above, and returns the final scores using *the same procedure* as questions 1-7.

*Note:* You should be able to reuse (or minorly change) the code from previous problems. Try to be DRY (don't repeat yourself)!

*Note 1:* Once adding the noise to the assignment scores, use the `np.clip` function to be sure each assignment retains a score between 0% and 100%.

*Note 2:* To check your work -- what would you expect the difference between the actual scores and noisy scores to be, on average?

In [36]:
df = get_prop_df(grades)

for col in columns['projects']:
    project_scores.append()

project_scores = sum(df['project'/len(columns['project'])

3      0.900989
4      0.648927
5      0.902117
6      0.794718
7      0.948493
         ...   
96     0.888833
97     0.900596
98     0.880979
99     0.893520
100    0.884403
Length: 98, dtype: float64

In [41]:
#get dataframe with grade proportions for each assignment
def get_prop_df(grades): 
    grades = grades.replace({np.nan: 0})

    #get lab grades first
    processed = process_labs(grades)
    df = processed

    names = get_assignment_names(grades)
    #we already have lab grades
    del names['lab']
    cols = unwind(names)


    for i in cols:
        score = grades[i] / grades[i + ' - Max Points']
        df[i] = score


    return df


    

In [64]:
def unwind(columns):
    columns.values()
    cols = []
    
    for i in columns.values():
        for j in i:
            cols.append(j)
            
    return cols

def total_points_with_noise(grades):
    """
    total_points_with_noise takes in a dataframe like grades,
    adds noise to the assignments as described in notebook, and returns
    the total scores of each student calculated with noisy grades.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = total_points_with_noise(grades)
    >>> np.all((0 <= out) & (out <= 1))
    True
    >>> 0.7 < out.mean() < 0.9
    True
    """

    props = get_prop_df(grades)
    
    num_rows = props.shape[0]
    num_cols = props.shape[1]
    random_amt = pd.DataFrame(np.random.normal(0, 0.02, size=(num_rows, num_cols)))
    
    
    noise = get_prop_df(grades).copy()
    columns = get_assignment_names(grades)
    n = 0
    
    for col in unwind(columns):
        noise[col] = np.clip(noise[col] + random_amt[n], 0, 100)
        n += 1
        
        
    #total grade calculation for noise
    #lab total
    lab_scores = lab_total(props)
    
    # project total
    project_scores = []
    for col in columns['project']:
        project_scores.append(noise[col])
    project_total = sum(project_scores)/len(columns['project'])
    
    # checkpoint total
    chk_scores = []
    for col in columns['checkpoint']:
        chk_scores.append(noise[col])
    chk_total = sum(chk_scores)/len(columns['checkpoint'])

    # discussion total
    disc_scores = []
    for col in columns['disc']:
        disc_scores.append(noise[col])
    disc_total = sum(disc_scores)/len(columns['disc'])
    
    #midterm and final
    midterm_total = noise['Midterm']
    final_total = noise['Final']
    
    
    return (lab_scores * 0.2) + (project_total * 0.3) + (chk_total * 0.025) + (disc_total * 0.025) + (midterm_total * 0.15) + (final_total * 0.3)

In [68]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = total_points_with_noise(grades)
#np.all((0 <= out) & (out <= 1))

0.7 < out.mean() < 0.9


True

In [29]:
len(get_assignment_names(grades))

6

### Short-answer questions (hard-coded)

Use your functions from above to understanding the data and answer the following questions. The function below should return **hard-coded values**. It should not compute anything!

**Question 10**

Create a function `short_answer` of zero variables that returns (hard-coded) answers to the following question in a list:
0. For the class on average, what is the difference between students' scores (`total_points`) and their scores with noise (`total_points_with_noise`)? (Remark: plot the distribution of differences; does this align with what you know about binomial distributions?)
1. What percentage of the class only sees their grade change at most (but not including) $\pm 0.01$?
2. What is the 95% confidence interval for the statistic above? (see [DSC10](https://www.inferentialthinking.com/chapters/13/3/Confidence_Intervals.html) and use `np.percentile`)
3. What proportion of the class sees a change in their letter grade?
4. The assumption behind the model in Question 9 is that:
    - The (observed) gradebook well represents the true population of students,
    - The noisy scores represent other possible observations drawn from the true population of students.
    - Answer `True` or `False`

# Congratulations, you finished the project!

### Before you submit:
* Be sure you run the doctests on all your code in project01.py

### To submit:
* **Upload the .py file to gradescope**